In [ ]:
# Import system libraries
import os
import sys

# Import data manipulation libraries
import pandas as pd
from shapely.geometry import Polygon

# Import geospatial libraries
import geopandas as gpd
import pygrib

# Import API library
import cdsapi # cds api client can be pip installed

# Data request from API 
Use the cdsapi client to retrieve data in GRIB format from CEMS. 

Note, requires API key in config folder in the file $HOME/.cdsapirc. See https://cds.climate.copernicus.eu/api-how-to. Mine is included in the conda environment.

In [ ]:
c = cdsapi.Client()
c.retrieve(
    'cems-fire-seasonal',
    {
        'format': 'grib',
        'variable': 'fire_weather_index',
        'release_version': '5',
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020', '2021', '2022',
            '2023',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': '01',
        'leadtime_hour': '12',
        'area': [
            89.95, -179.95, -89.95,
            179.95,
        ],
    },
    'download.grib')

In [ ]:
# Following code should repat the above for other leads. 
# They are equal to 12 + 24*n, so 12, 36, 60...

# Clean and merge the downloaded GRIB files


In [ ]:
# Open the GRIB file
grib_file = 'assets\data\CEMS_indicators\download.grib'
grbs = pygrib.open(grib_file)

# Read the geospatial data from the GRIB file
gdf_list = []
for grb in grbs:
    grb_data = grb.values
    lats, lons = grb.latlons()
    
    # Create a GeoDataFrame with the geospatial data
    geometry = [shapely.geometry.Point(lon, lat) for lat, lon in zip(lats.flatten(), lons.flatten())]
    gdf = gpd.GeoDataFrame(geometry=geometry, data=grb_data.flatten(), columns=['data'])
    
    gdf_list.append(gdf)

# Close the GRIB file
grbs.close()

# Concatenate GeoDataFrames
gdf = gpd.concat(gdf_list)

# Now, gdf is a GeoDataFrame containing geospatial data from the GRIB file